# Create Mnemonic from Entropy
## 1. Create Entropy
### What is Entropy?
Entropy is a sequence of random data used as the foundation for creating a secure mnemonic phrase. In the context of BIP-39, it is represented as a binary string generated with a specific length (e.g., 128, 160, 192, 224, or 256 bits). The higher the entropy, the more secure the mnemonic phrase will be.

In [3]:
import secrets

# Define strength (in bits) for the entropy
strength = 128  # You can set this to 128, 160, 192, 224, or 256
strength_bytes = strength // 8

# Generate entropy using the secrets module
entropy = secrets.token_bytes(strength_bytes)

# Print the generated entropy in hexadecimal format
print(f'Created entropy: {entropy.hex()} ({len(entropy)} bytes)')

Created entropy: 9fcf0dcecdd9d986a71b09b8cb467065 (16 bytes)


## 2. Create Mnemonic
### 2-1. Load wordlist
#### What is a Wordlist?
The wordlist is a predefined list of 2048 unique words as specified in the BIP-39 standard. These words are used to convert binary data (entropy + checksum) into a human-readable mnemonic phrase. Each word corresponds to an 11-bit binary number.

In [4]:
wordlist = []
with open("english.txt", "r", encoding="utf-8") as f:
    wordlist = [w.strip() for w in f.readlines()]

print('Successfully loaded wordlist')
wordlist

Successfully loaded wordlist


['abandon',
 'ability',
 'able',
 'about',
 'above',
 'absent',
 'absorb',
 'abstract',
 'absurd',
 'abuse',
 'access',
 'accident',
 'account',
 'accuse',
 'achieve',
 'acid',
 'acoustic',
 'acquire',
 'across',
 'act',
 'action',
 'actor',
 'actress',
 'actual',
 'adapt',
 'add',
 'addict',
 'address',
 'adjust',
 'admit',
 'adult',
 'advance',
 'advice',
 'aerobic',
 'affair',
 'afford',
 'afraid',
 'again',
 'age',
 'agent',
 'agree',
 'ahead',
 'aim',
 'air',
 'airport',
 'aisle',
 'alarm',
 'album',
 'alcohol',
 'alert',
 'alien',
 'all',
 'alley',
 'allow',
 'almost',
 'alone',
 'alpha',
 'already',
 'also',
 'alter',
 'always',
 'amateur',
 'amazing',
 'among',
 'amount',
 'amused',
 'analyst',
 'anchor',
 'ancient',
 'anger',
 'angle',
 'angry',
 'animal',
 'ankle',
 'announce',
 'annual',
 'another',
 'answer',
 'antenna',
 'antique',
 'anxiety',
 'any',
 'apart',
 'apology',
 'appear',
 'apple',
 'approve',
 'april',
 'arch',
 'arctic',
 'area',
 'arena',
 'argue',
 'arm',
 

### 2-2. Calculate hash of entropy
#### Why Calculate a Hash?
The hash of the entropy (using SHA-256) is used to generate a checksum. This checksum ensures the integrity of the entropy and adds an additional layer of verification during mnemonic phrase generation.

In [5]:
import hashlib

entropy_hash = hashlib.sha256(entropy).hexdigest()
print(f'Calculated sha256 hash of entropy: {entropy_hash}')

Calculated sha256 hash of entropy: 2a75d4e4495095822f26edcce6b7cb5798985b319f09b2b908d447860b6c9f64


### 2-3. Create Entropy bits + Checksum bits for Creating Mnemonic
#### What are Checksum Bits?
Checksum bits are derived from the SHA-256 hash of the entropy. The number of checksum bits is determined by the length of the entropy divided by 32. These bits are appended to the entropy bits to form the full binary data used for mnemonic generation.

In [10]:
# Convert the entropy to binary bits.
# The entropy is converted from bytes to an integer, then to a binary string.
# The length is padded to match the number of bits in the entropy.
entropy_bits = b = bin(int.from_bytes(entropy, byteorder="big"))[2:].zfill(len(entropy) * 8)
print(f'Entroy bits: {entropy_bits} (len: {len(entropy_bits)})')


# Compute the SHA-256 hash of the entropy, which is used to derive the checksum.
# The hash is assumed to be in hexadecimal format, so it's converted to binary.
entropy_hash_bits = bin(int(entropy_hash,16))[2:].zfill(256)
print(f'Entropy hash bits: {entropy_hash_bits}')

# Extract the first (entropy length / 32) bits of the hash to use as the checksum.
# This ensures the checksum length is proportional to the entropy length.
checksum_bits = entropy_hash_bits[: len(entropy) * 8 // 32]
print(f'Checksum bits: {checksum_bits} (len: {len(checksum_bits)})')

# Concatenate the entropy bits with the checksum bits.
# This forms the final bit sequence that will be used to generate the mnemonic words.
bits = entropy_bits + checksum_bits
print(f'Bits: {bits} (len: {len(bits)})')

Entroy bits: 10011111110011110000110111001110110011011101100111011001100001101010011100011011000010011011100011001011010001100111000001100101 (len: 128)
Entropy hash bits: 0010101001110101110101001110010001001001010100001001010110000010001011110010011011101101110011001110011010110111110010110101011110011000100110000101101100110001100111110000100110110010101110010000100011010100010001111000011000001011011011001001111101100100
Checksum bits: 0010 (len: 4)
Bits: 100111111100111100001101110011101100110111011001110110011000011010100111000110110000100110111000110010110100011001110000011001010010 (len: 132)


### 2-4. Create mnemonic from entropy + checksum bits 
#### How is the Mnemonic Phrase Created?
- **Split the combined bitstring**: The combined entropy and checksum bitstring is divided into fixed-length 11-bit segments.
- **Convert binary to integer**: Each 11-bit segment is interpreted as a binary number and converted to an integer using `int(segment, 2)`. This integer serves as an index for the BIP-39 wordlist.
- **Map integers to words**: The integers derived from the 11-bit segments are used to select words from the wordlist. These words collectively form the mnemonic phrase.
- **Construct the mnemonic phrase**: The selected words are appended to form a human-readable and secure mnemonic phrase.

In [11]:
result = []
for i in range(len(bits) // 11):
    idx = int(bits[i * 11 : (i + 1) * 11], 2)
    print(f'Calculated idx: {idx}')
    word = wordlist[idx]
    print(f'Word at {idx}: {word}')
    result.append(word)

print(f'Successfully created mnemonic: {result}')

Calculated idx: 1278
Word at 1278: panther
Calculated idx: 963
Word at 963: journey
Calculated idx: 925
Word at 925: inhale
Calculated idx: 1245
Word at 1245: option
Calculated idx: 1260
Word at 1260: outside
Calculated idx: 1562
Word at 1562: select
Calculated idx: 1251
Word at 1251: organ
Calculated idx: 777
Word at 777: gentle
Calculated idx: 1478
Word at 1478: rhythm
Calculated idx: 721
Word at 721: focus
Calculated idx: 1248
Word at 1248: orchard
Calculated idx: 1618
Word at 1618: ski
Successfully created mnemonic: ['panther', 'journey', 'inhale', 'option', 'outside', 'select', 'organ', 'gentle', 'rhythm', 'focus', 'orchard', 'ski']


## 3. Validate Mnemonic
### What is Mnemonic Validation?
Mnemonic validation ensures that a given mnemonic phrase is valid according to the BIP-39 standard. This involves checking:
1. The mnemonic length is appropriate (12, 15, 18, 21, or 24 words).
2. Each word exists in the BIP-39 wordlist.
3. The checksum derived from the mnemonic matches the original entropy's checksum.

In [12]:
def validate_mnemonic(mnemonic_list: list, wordlist: list) -> bool:
    # Check if the number of words is valid
    if len(mnemonic_list) not in [12, 15, 18, 21, 24]:
        print("Invalid mnemonic length")
        return False

    # Convert words to binary indexes
    try:
        idx = map(lambda x: bin(wordlist.index(x))[2:].zfill(11), mnemonic_list)
        b = "".join(idx)
    except ValueError:
        print("Mnemonic contains invalid words")
        return False

    # Separate the binary string into entropy and checksum parts
    l = len(b)  # Total bit length
    d = b[: l // 33 * 32]  # Entropy bits
    h = b[-l // 33 :]  # Checksum bits

    # Recalculate the checksum from entropy
    nd = int(d, 2).to_bytes(l // 33 * 4, byteorder="big")
    nh = bin(int(hashlib.sha256(nd).hexdigest(), 16))[2:].zfill(256)[: l // 33]

    # Validate the checksum
    if h == nh:
        print("Mnemonic is valid")
        return True
    else:
        print("Invalid checksum")
        return False

In [13]:
# Example Usage
print(f'Validate mnemonic we created before: {result}')
is_valid = validate_mnemonic(result, wordlist)
if is_valid == True:
    print(f'The mnemonic we created is valid')

Validate mnemonic we created before: ['panther', 'journey', 'inhale', 'option', 'outside', 'select', 'organ', 'gentle', 'rhythm', 'focus', 'orchard', 'ski']
Mnemonic is valid
The mnemonic we created is valid
